In [7]:
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

In [8]:
from src.utils.load_dataframe import load_dataframe , TimeGranularity

In [9]:
df = load_dataframe(sheet_name=TimeGranularity.MIN_15)

In [10]:
print(df.columns)

Index(['utc_timestamp', 'cet_cest_timestamp',
       'AT_load_actual_entsoe_transparency',
       'AT_load_forecast_entsoe_transparency', 'AT_price_day_ahead',
       'AT_solar_generation_actual', 'AT_wind_onshore_generation_actual',
       'BE_load_actual_entsoe_transparency',
       'BE_load_forecast_entsoe_transparency',
       'DE_load_actual_entsoe_transparency',
       'DE_load_forecast_entsoe_transparency', 'DE_solar_capacity',
       'DE_solar_generation_actual', 'DE_solar_profile', 'DE_wind_capacity',
       'DE_wind_generation_actual', 'DE_wind_profile',
       'DE_wind_offshore_capacity', 'DE_wind_offshore_generation_actual',
       'DE_wind_offshore_profile', 'DE_wind_onshore_capacity',
       'DE_wind_onshore_generation_actual', 'DE_wind_onshore_profile',
       'DE_50hertz_load_actual_entsoe_transparency',
       'DE_50hertz_load_forecast_entsoe_transparency',
       'DE_50hertz_solar_generation_actual',
       'DE_50hertz_wind_generation_actual',
       'DE_50hertz_wind_

In [11]:
df.columns
df['utc_timestamp'] = pd.to_datetime(df['utc_timestamp'], utc=True)
df.set_index('utc_timestamp', inplace=True)

In [12]:
# Seleccionar la columna que se desea analizar, por ejemplo, 'AT_load_actual_entsoe_transparency'
series = df['AT_load_actual_entsoe_transparency'].dropna()

In [13]:
# Dividir en datos de entrenamiento y prueba
start_time = pd.to_datetime('2018-12-01 00:00:00').tz_localize('UTC')
train_data = series[:start_time]
test_data = series[start_time:]

In [14]:
import itertools
# Definir la cuadrícula de parámetros
p = d = q = range(0, 4)
pdq = list(itertools.product(p, d, q))

lowest_aic = np.inf
best_pdq = None


In [16]:
# Búsqueda de cuadrícula
for param in pdq:
    try:
        model = SARIMAX(train_data, order=param, seasonal_order=(1, 1, 1, 24))
        model_fit = model.fit()

        # Guardar parámetros del modelo con el AIC más bajo
        if model_fit.aic < lowest_aic:
            lowest_aic = model_fit.aic
            best_pdq = param
            best_model = model_fit
    except:
        continue

/Users/alejandrovelazco/work/master-tesis/.venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency 15min will be used.
  self._init_dates(dates, freq)
/Users/alejandrovelazco/work/master-tesis/.venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency 15min will be used.
  self._init_dates(dates, freq)


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            3     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  8.74672D+00    |proj g|=  2.04756D-01


 This problem is unconstrained.



At iterate    5    f=  8.64477D+00    |proj g|=  1.92632D-01


/Users/alejandrovelazco/work/master-tesis/.venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency 15min will be used.
  self._init_dates(dates, freq)
/Users/alejandrovelazco/work/master-tesis/.venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency 15min will be used.
  self._init_dates(dates, freq)
/Users/alejandrovelazco/work/master-tesis/.venv/lib/python3.12/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'


...

 This problem is unconstrained.



At iterate    5    f=  8.32591D+00    |proj g|=  4.09860D-03


/Users/alejandrovelazco/work/master-tesis/.venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency 15min will be used.
  self._init_dates(dates, freq)
/Users/alejandrovelazco/work/master-tesis/.venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency 15min will be used.
  self._init_dates(dates, freq)
 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            5     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  8.74567D+00    |proj g|=  1.03311D+00


/Users/alejandrovelazco/work/master-tesis/.venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency 15min will be used.
  self._init_dates(dates, freq)
/Users/alejandrovelazco/work/master-tesis/.venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency 15min will be used.
  self._init_dates(dates, freq)


In [ ]:
import joblib

print('Best SARIMA parameters:', best_pdq)
joblib.dump(best_model, f'modelos/SARIMA/model_{best_pdq}.joblib')

Best SARIMA parameters: None


NameError: name 'best_model' is not defined

In [ ]:
# Cargar el mejor modelo
model = joblib.load(f'modelos/SARIMA/model_{best_pdq}.joblib')

In [ ]:
# Realizar predicciones para un rango de tiempo dado
end_time = pd.to_datetime('2018-12-02 00:00:00').tz_localize('UTC')
predictions = model.get_prediction(start=start_time, end=end_time)

In [ ]:

# Calcular el MSE
mse = mean_squared_error(test_data[start_time:end_time], predictions.predicted_mean)
print('Mean Squared Error: ', mse)

# Graficar predicciones vs valores reales
plt.figure(figsize=(10, 6))
plt.plot(predictions.predicted_mean, label='Predictions')
plt.plot(test_data[start_time:end_time], label='Actual values')
plt.title('Predictions vs Actual values')
plt.xlabel('Time')
plt.ylabel('Energy demand')
plt.legend()
plt.show()